# 🚀 VTrackAI Studio - GPU Backend (SAM3)

**Connect local frontend to Colab GPU for 10x faster SAM3 tracking!**

**Requirements:**
- Colab GPU runtime (T4/A100)
- Hugging Face account (free)

**Steps:**
1. Runtime → Change runtime type → T4 GPU
2. Run all cells (Ctrl+F9)
3. Copy **Cloudflare URL** (https://xxx.trycloudflare.com)
4. Local: `VITE_API_URL=<url> npm run dev`

## 1. Check GPU

In [ ]:
import torch
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU - Runtime > Change runtime type > GPU")

🔥 CUDA Available: True
🎮 GPU: Tesla T4
📊 VRAM: 15.8 GB


## 2. Clean Install + Dependencies

In [ ]:
!rm -rf /content/sam3 /content/vtrack-ai-studio
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart opencv-python librosa huggingface_hub
!pip install -q cloudflared nest-asyncio
print("✅ All dependencies ready")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sam3 0.1.0 requires numpy<2,>=1.26, but you have numpy 2.0.2 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 3.6 MB/s eta 0:00:00
✅ All dependencies ready


## 3. Install SAM3 + VTrackAI

In [ ]:
!git clone https://github.com/facebookresearch/sam3
!pip install -e ./sam3 --quiet
!git clone https://github.com/Amitro123/vtrack-ai-studio
%cd vtrack-ai-studio/backend
!pip install -r requirements.txt --quiet
print("✅ SAM3 + VTrackAI installed")

Cloning into 'sam3'...
remote: Enumerating objects: 628, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 628 (delta 29), reused 21 (delta 21), pack-reused 590 (from 2)
Receiving objects: 100% (628/628), 58.95 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (100/100), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sam3 (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2;

## 4. Hugging Face Login + SAM3 Checkpoint

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import os

# Auto-login from Colab secrets
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

print("✅ HF token loaded from secrets")


In [ ]:
# Cell 4.5: Download SAM3 checkpoints OFFICIAL script
import os
os.chdir('/content')

# Official SAM3 download script
!wget https://raw.githubusercontent.com/facebookresearch/sam3/main/scripts/download_ckpts.py
!python download_ckpts.py --model_size large --output_dir /content/vtrack-ai-studio/backend/checkpoints/sam3

print("✅ Official SAM3 checkpoints downloaded!")
!ls -la /content/vtrack-ai-studio/backend/checkpoints/sam3/


In [ ]:
from huggingface_hub import hf_hub_download
import os

os.makedirs("checkpoints/sam3", exist_ok=True)

# SAM3 base model (public)
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="checkpoints/sam3_hiera_tiny.pt",  # Tiny model - 100% public
    local_dir="./checkpoints/sam3"
)
print(f"✅ Tiny SAM3: {checkpoint_path}")


EntryNotFoundError: 404 Client Error. (Request ID: Root=1-6950282a-0592350621ce831a1611b74c;9e66f7a3-9897-49a0-94d1-6763fca4309a)

Entry Not Found for url: https://huggingface.co/facebook/sam3/resolve/main/checkpoints/sam3_hiera_tiny.pt.

## 5. 🚀 Launch GPU Backend (Cloudflare Tunnel)

In [ ]:
import nest_asyncio
import uvicorn
import subprocess
import threading
import time
import requests

nest_asyncio.apply()

# Start FastAPI backend
def start_backend():
    uvicorn.run("server:app", host="0.0.0.0", port=8000, log_level="info")

backend_thread = threading.Thread(target=start_backend, daemon=True)
backend_thread.start()
time.sleep(8)  # Wait for startup

print("🚀 Backend started on http://localhost:8000")
print("⏳ Starting Cloudflare tunnel...")

# Cloudflare tunnel (FREE + stable)
!nohup cloudflared tunnel --url http://localhost:8000 > cloudflared.log 2>&1 &
time.sleep(8)

# Get Cloudflare URL from logs
!grep -o 'https://[^ ]*\.trycloudflare\.com' cloudflared.log || echo "🔍 Check cloudflared.log for URL"

print("\n" + "="*60)
print("🎉 VTrackAI GPU Backend LIVE!")
print("📡 Look for Cloudflare URL above (https://xxx.trycloudflare.com)")
print("="*60)
print("👉 Local frontend: VITE_API_URL=<cloudflare-url> npm run dev")

# Keep alive
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("\n👋 Backend stopped")

## 🔍 Health Check (Run after backend starts)

In [ ]:
# Test backend health
import requests
r = requests.get("http://localhost:8000/api/health")
print("Health:", r.json())
print("✅ GPU tracking ready! 🚀")